In [14]:
import os
import json
import datetime
from dateutil import parser
import glob
import json
from dataclasses import dataclass, field
from dacite import from_dict, Config

CREDENTIALS_FILE = "./creds.txt"
USERDIR = "datasets/users"
QUERY_RESULTS = "queried_users_ia.jsonl"
TWEET_QUERY_RESULTS = "queried_users_tweets_ia.jsonl"
# control = True
# if control:
#     QUERY_RESULTS = "control_" + QUERY_RESULTS
#     TWEET_QUERY_RESULTS = "control_" + TWEET_QUERY_RESULTS

# TT_hongkong.nd json
# First Tweet retrieved Friday, August 9, 2019 8:33:46 PM EST
# Result_type: recent

In [5]:
@dataclass
class Tweet:
    id: int
    text: str
    created_at: str
    lang: str
    source: str
    retweeted: bool

@dataclass
class User:
    id: int
    screen_name: str
    name: str
    description: str
    location: str
    tweets: list[Tweet] = field(default_factory=list)

In [19]:
users = {}
with open(QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        users[data["id"]] = data
        
users_control = {}
with open("control_" + QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        users_control[data["id"]] = data

print("% Accounts deleted or protected since")
print("EXPERIMENT:")
print(1-sum([result["found"] and not result["protected"] for result in users.values()])/float(len(users)))
print("CONTROL:")
print(1-sum([result["found"] and not result["protected"] for result in users_control.values()])/float(len(users_control)))

% Accounts deleted or protected since
EXPERIMENT:
0.22587570621468922
CONTROL:
0.1375538329026701


In [20]:
print(TWEET_QUERY_RESULTS)

tweets = {}
rts = 0
with open(TWEET_QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        if data["found"] and data["text"].startswith("RT @"):
            rts += 1
            continue
        tweets[data["id"]] = data
        
rts_control = 0
tweets_control = {}
with open("control_" + TWEET_QUERY_RESULTS, "r") as f:
    for line in f:
        line = line.strip()
        data = json.loads(line)
        if data["found"] and data["text"].startswith("RT @"):
            rts_control += 1
            continue
        tweets_control[data["id"]] = data

# TODO: CONTROL FOR DELETED RTS
print("% Tweets of public users deleted since")
print("EXPERIMENT:")
print(1-sum([result["found"] for result in users.values()])/float(len(users)))
print("CONTROL:")
print(1-sum([result["found"] for result in users_control.values()])/float(len(users_control)))

# print("==============")
# print("% RTs that still exist today")
# print("EXPERIMENT:")
# print(rts / float(rts + len(tweets)))
# print("CONTROL:")
# print(rts_control / float(rts_control + len(tweets)))

queried_users_tweets_ia.jsonl
% Tweets of public users deleted since
EXPERIMENT:
0.1403954802259887
CONTROL:
0.09276485788113698
